# ETL Process

#### The data file contains all json and log files where i need to go through it and need to insert into Database

In [1]:
import os
import glob
import pandas as pd
from create_tables import *
from sql_queries import *
import psycopg2

In [2]:
conn=psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=postgres password=macbook")
cur=conn.cursor()
conn.set_session(autocommit=True)

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
#         print(files)
        files = glob.glob(os.path.join(root,'*.json'))
        print(files)
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

In [4]:
song_files=get_files("data/song_data")

[]
[]
[]
['data/song_data/A/A/A/TRAAAEF128F4273421.json', 'data/song_data/A/A/A/TRAAARJ128F9320760.json', 'data/song_data/A/A/A/TRAAAFD128F92F423A.json', 'data/song_data/A/A/A/TRAAAPK128E0786D96.json', 'data/song_data/A/A/A/TRAAAVO128F93133D4.json', 'data/song_data/A/A/A/TRAAABD128F429CF47.json', 'data/song_data/A/A/A/TRAAAAW128F429D538.json', 'data/song_data/A/A/A/TRAAADZ128F9348C2E.json', 'data/song_data/A/A/A/TRAAAVG12903CFA543.json', 'data/song_data/A/A/A/TRAAAMO128F1481E7F.json', 'data/song_data/A/A/A/TRAAAMQ128F1460CD3.json']
['data/song_data/A/A/C/TRAACOW128F933E35F.json', 'data/song_data/A/A/C/TRAACPE128F421C1B9.json', 'data/song_data/A/A/C/TRAACTB12903CAAF15.json', 'data/song_data/A/A/C/TRAACFV128F935E50B.json', 'data/song_data/A/A/C/TRAACIW12903CC0F6D.json', 'data/song_data/A/A/C/TRAACVS128E078BE39.json', 'data/song_data/A/A/C/TRAACER128F4290F96.json', 'data/song_data/A/A/C/TRAACSL128F93462F4.json', 'data/song_data/A/A/C/TRAACHN128F1489601.json', 'data/song_data/A/A/C/TRAACLV

In [115]:
filepath=song_files[10]

In [116]:
filepath

'/Users/nikhil/Desktop/learning/Data_engineering_Udacity/Part 1 Data modeling/Project 1 Postgres/data/song_data/A/A/A/TRAAAMQ128F1460CD3.json'

In [117]:
df=pd.read_json(filepath,lines=True)
df

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARD0S291187B9B7BF5,NaN,Ohio,NaN,Rated R,114.78159,1,SOMJBYD12A6D4F8557,Keepin It Real (Skit),0


In [118]:
df.columns.values

array(['artist_id', 'artist_latitude', 'artist_location',
       'artist_longitude', 'artist_name', 'duration', 'num_songs',
       'song_id', 'title', 'year'], dtype=object)

In [119]:
artist_id,artist_latitude, artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year=df.values[0]


In [120]:
song_data=[song_id,title,artist_id,year,duration]
song_data

['SOMJBYD12A6D4F8557',
 'Keepin It Real (Skit)',
 'ARD0S291187B9B7BF5',
 0,
 114.78159]

In [121]:
# list_insert

In [122]:
cur.execute(songs_insert,song_data)

## Insert into the artist table 

### The content needs to be there is 
(artist_id, name, location, lattitude, longitude)

In [87]:
data=[artist_id,artist_name,artist_location,artist_latitude,artist_longitude]
data

['ARD0S291187B9B7BF5', 'Rated R', 'Ohio', nan, nan]

In [88]:
cur.execute(artists_insert,data)

In [89]:
cur.execute("select * from artists")

In [90]:
print(cur.fetchall())

[('ARD0S291187B9B7BF5', 'Rated R', 'Ohio', nan, nan)]


# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [91]:
log_files=get_files("data/log_data/")

In [92]:
filepath=log_files[0]

In [93]:
filepath

'/Users/nikhil/Desktop/learning/Data_engineering_Udacity/Part 1 Data modeling/Project 1 Postgres/data/log_data/2018/11/2018-11-11-events.json'

In [94]:
df=pd.read_json(filepath,lines=True)

In [95]:
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Frumpies,Logged In,Anabelle,F,0,Simpson,134.47791,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,Fuck Kitty,200,1541903636796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
1,Kenny G with Peabo Bryson,Logged In,Anabelle,F,1,Simpson,264.75057,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,By The Time This Night Is Over,200,1541903770796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
2,Biffy Clyro,Logged In,Anabelle,F,2,Simpson,189.83138,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,God & Satan,200,1541904034796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
3,None,Logged In,Lily,F,0,Burns,NaN,free,"New York-Newark-Jersey City, NY-NJ-PA",GET,Home,1.540621e+12,456,None,200,1541910841796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32
4,HIM,Logged In,Lily,F,1,Burns,212.06159,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540621e+12,456,Beautiful,200,1541910973796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32


###     INSERT INTO time
    (start_time, hour, day, week, month, year, weekday)


#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [96]:
df=df[df.page=='NextSong']
df['time_data']=pd.to_datetime(df.ts,unit='ms')
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId,time_data
0,Frumpies,Logged In,Anabelle,F,0,Simpson,134.47791,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,Fuck Kitty,200,1541903636796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69,2018-11-11 02:33:56.796
1,Kenny G with Peabo Bryson,Logged In,Anabelle,F,1,Simpson,264.75057,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,By The Time This Night Is Over,200,1541903770796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69,2018-11-11 02:36:10.796
2,Biffy Clyro,Logged In,Anabelle,F,2,Simpson,189.83138,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,God & Satan,200,1541904034796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69,2018-11-11 02:40:34.796
4,HIM,Logged In,Lily,F,1,Burns,212.06159,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540621e+12,456,Beautiful,200,1541910973796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32,2018-11-11 04:36:13.796
5,Matmos,Logged In,Joseph,M,0,Gutierrez,1449.11628,free,"Columbia, SC",PUT,NextSong,1.540809e+12,284,Supreme Balloon,200,1541911006796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3...",75,2018-11-11 04:36:46.796


In [97]:
t=df.time_data
for i in t:
    print(i.weekday_name)
    break

Sunday


In [98]:
time_data=[]
column_labels ={'start_time':[], 'hour':[], 'day':[], 'week':[], 'month':[], 'year':[], 'weekday':[]}

for i in t:
    column_labels['start_time'].append(i)
    column_labels['hour'].append(i.hour)
    column_labels['day'].append(i.day)
    column_labels['week'].append(i.week)
    column_labels['month'].append(i.month)
    column_labels['year'].append(i.year)
    column_labels['weekday'].append(i.weekday_name)

In [99]:
time_df=pd.DataFrame(column_labels,columns=['start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday'])

In [100]:
time_df.head()

,start_time,hour,day,week,month,year,weekday
0,2018-11-11 02:33:56.796,2,11,45,11,2018,Sunday
1,2018-11-11 02:36:10.796,2,11,45,11,2018,Sunday
2,2018-11-11 02:40:34.796,2,11,45,11,2018,Sunday
3,2018-11-11 04:36:13.796,4,11,45,11,2018,Sunday
4,2018-11-11 04:36:46.796,4,11,45,11,2018,Sunday


In [101]:
for i,row in time_df.iterrows():
    cur.execute(time_insert,list(row))
cur.execute("select * from time where start_time ='2018-11-11 02:33:56.796'")


In [102]:
cur.execute("select * from time")
print(cur.fetchall())

[(datetime.date(2018, 11, 11), 2, 11, 45, 11, 2018, 'Sunday')]


## User table 

columns needed are 
**users** - users in the app
- user_id (INT) PRIMARY KEY: ID of user
- first_name (TEXT) NOT NULL: Name of user
- last_name (TEXT) NOT NULL: Last Name of user
- gender (TEXT): Gender of user {M | F}
- level (TEXT): User level {free | paid}

In [103]:
user_df=df[['userId','firstName','lastName','gender','level']]
user_df.head()

,userId,firstName,lastName,gender,level
0,69,Anabelle,Simpson,F,free
1,69,Anabelle,Simpson,F,free
2,69,Anabelle,Simpson,F,free
4,32,Lily,Burns,F,free
5,75,Joseph,Gutierrez,M,free


In [104]:
for i, row in user_df.iterrows():
    cur.execute(user_insert,list(row))


In [105]:
cur.execute("select * from users")
for i in cur.fetchall():
    print(i)

(69, 'Anabelle', 'Simpson', 'F', 'free')
(32, 'Lily', 'Burns', 'F', 'free')
(75, 'Joseph', 'Gutierrez', 'M', 'free')
(92, 'Ryann', 'Smith', 'F', 'free')
(49, 'Chloe', 'Cuevas', 'F', 'free')
(88, 'Mohammad', 'Rodriguez', 'M', 'free')
(96, 'Cierra', 'Finley', 'F', 'free')
(26, 'Ryan', 'Smith', 'M', 'free')
(44, 'Aleena', 'Kirby', 'F', 'paid')
(67, 'Colm', 'Santana', 'M', 'free')
(62, 'Connar', 'Moreno', 'M', 'free')
(14, 'Theodore', 'Harris', 'M', 'free')
(16, 'Rylan', 'George', 'M', 'free')
(80, 'Tegan', 'Levine', 'F', 'paid')
(35, 'Molly', 'Taylor', 'F', 'free')


## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

song_select = ("""
    SELECT song_id, artists.artist_id
    FROM songs JOIN artists ON songs.artist_id = artists.artist_id
    WHERE songs.title = %s
    AND artists.name = %s
    AND songs.duration = %s
""")


### songplay_insert="insert into songplays(songplay_id, start_time,user_id,level,song_id,artist_id,session_id, location,user_agent)values(%s,%s,%s,%s,%s,%s,%s,%s,%s) on conflict (songplay_id) do nothing"

In [106]:
for i,row in df.iterrows():
    cur.execute(song_select,(row.song,row.artist,row.length))
    
    result=cur.fetchone()
#     print(result)
    if result:
        songid,artistid=result
    else:
        songid,artistid=None,None
    song_data=(i,row.time_data,row.userId,row.level,songid,artistid,row.sessionId,row.location,row.userAgent)
    cur.execute(songplay_insert,song_data)
    conn.commit()
    

None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


###  Close the connections

In [107]:
cur.close()
conn.close()


# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.